## DESCRIPTION

_r.futures.*_ is an implementation of FUTure Urban-Regional
Environment Simulation (FUTURES) which is a model for multilevel
simulations of emerging urban-rural landscape structure. FUTURES
produces regional projections of landscape patterns using coupled
submodels that integrate nonstationary drivers of land change: per
capita demand (DEMAND submodel), site suitability (POTENTIAL submodel),
and the spatial structure of conversion events (PGA submodel).


### Submodels

  _DEMAND_
  DEMAND estimates the rate of per capita land consumption
  specific to each subregion. Projections of land consumption are based
  on extrapolations between historical changes in population
  and land conversion based on scenarios of future population growth.
  How to construct the per capita demand relationship for subregions depends
  on user's preferences and data availability.
  Land area conversion over time can be derived for the USA, e.g.
  from National Land Cover Dataset.
  A possible implementation of the DEMAND submodel is available as module
  _[r.futures.demand](r.futures.demand.html)_.

  _POTENTIAL_
  The POTENTIAL submodel uses site suitability modeling approaches
  to quantify spatial gradients of land development potential.
  The model uses multilevel logistic regression to
  account for hierarchical characteristics of the land use system
  (variation among jurisdictional structures) and
  account for divergent relationships between predictor and response variables.
  To generate a binary, developed-undeveloped response variable
  using a stratified-random sample,
  see module _[r.sample.category](r.sample.category.html)_.
  The coefficients for the statistical model that are used to
  calculate the value of development potential can be derived in different ways,
  one possible implementation using R will be available
  as module _r.futures.potential_.
  One of the predictor variables is development pressure (computed using
  _[r.futures.devpressure](r.futures.devpressure.html)_)
  which is updated each step and thus creates positive feedback
  resulting in new development attracting even more development.
  

  _PGA_
  Patch-Growing Algorithm is a stochastic algorithm, which
  simulates undeveloped to developed land change by iterative site selection
  and a contextually aware region growing mechanism.
  Simulations of change at each time step feed development pressure back
  to the POTENTIAL submodel, influencing site suitability for the next step.
  PGA is implemented in [r.futures.pga](r.futures.pga.html).






### Intro to Jupyter Notebook

In [ ]:
# This is a quick introduction into Jupyter Notebook.
# Python code can be executed like this:
a = 6
b = 7
c = a * b
print("Answer is", c)
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

### Setting up GRASS GIS session


In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "data/grassdata", "futures_triangle_nc", "workshop_US_IALE_2019")
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

## Initial steps and data preparation
First we will set the computational region of our analyses to an extent covering our study area that aligns with our base landuse rasters. 

In [ ]:
gs.run_command('g.region', raster='landuse_2011', flags='p')

The FUTURES model uses the locations of development and new development to predict when and where new urban growth will occur. This requires longitudinal data on urban land use. We will derive urbanized areas from NLCD dataset for year 1992, 2001, 2006 and 2011 by extracting categories category 21 - 24 into a new binary map where developed is 1, undeveloped 0 and NULL (no data). The NULL area is important in simulating urban growth as it precludes these are from becoming development. For this exercise we assume that water, wetlands and protected areas can not be developed, setting them to NULL.

First we will convert protected areas from vector to raster. We set NULLs to zeros (for simpler raster algebra expression in the next step) using r.null. Copy line by line into GUI command console and press Enter after each command.

In [ ]:
gs.run_command('v.to.rast', input='protected_areas', output='protected_areas', use='val')
gs.run_command('r.null', map='protected_areas', null=0)

And then create rasters of developed/undeveloped areas using raster algebra:

In [ ]:
gs.mapcalc("urban_1992 = if(landuse_1992 >= 20 && landuse_1992 <= 24, 1, if(landuse_1992 == 11 || landuse_1992 >= 90 || protected_areas, null(), 0))")
gs.mapcalc("urban_2001 = if(landuse_2001 >= 21 && landuse_2001 <= 24, 1, if(landuse_2001 == 11 || landuse_2001 >= 90 || protected_areas, null(), 0))")
gs.mapcalc("urban_2006 = if(landuse_2006 >= 21 && landuse_2006 <= 24, 1, if(landuse_2006 == 11 || landuse_2006 >= 90 || protected_areas, null(), 0))")
gs.mapcalc("urban_2011 = if(landuse_2011 >= 21 && landuse_2011 <= 24, 1, if(landuse_2011 == 11 || landuse_2011 >= 90 || protected_areas, null(), 0))")

FUTURES is a multi-level modelling framework meaning that we model population projections for subregions and assume that these subregions are influenced differently by the factors that influence where development happens. This requires defining subregions for model implementation. For this exercise we use the county level, which is the finest scale of population data that is available to us. We therefore need to convert vectors of counties to a raster with the values of the FIPS attribute which links to population file:


In [ ]:
gs.run_command('v.to.rast', input='counties', type='area', use='attr', attribute_column='value', output='counties')

Before further steps, we will set our working directory so that the input population files and text files we are going to create in the following section are saved in one directory and easily accessible. You can do that from menu Settings → GRASS working environment → Change working directory. Select (or create) a directory and move there the downloaded files population_projection.csv and population_trend.csv.

## Potential submodel
Module r.futures.potential implements the POTENTIAL submodel as a part of FUTURES land change model. POTENTIAL is implemented using a set of coefficients that relate a selection of site suitability factors to the probability of a place becoming developed. This is implemented using the parameter table in combination with maps of those site suitability factors (mapped predictors). The coefficients are obtained by conducting multilevel logistic regression in R with package lme4 where the coefficients may vary by county. The best model is selected automatically using dredge function from package MuMIn.

First, we will derive possible predictors of urban growth using GRASS modules.

### Predictors
##### Slope
Slope possibly influences urban growth as steep areas are difficult to build on. We will derive slope in degrees from digital elevation model using r.slope.aspect module:

In [ ]:
gs.run_command('r.slope.aspect', elevation="elevation_30m", slope="slope")

#### Distance from lakes/rivers
Distance to lakes or rivers may contribute to an amenity draw where people would like to build a house in a location where they can view water. First we will extract category Open Water from 2011 NLCD dataset:

In [ ]:
gs.mapcalc("water = if(landuse_2011 == 11, 1, null())")

Then we compute the distance to water with module r.grow.distance and set color table to shades of blue:

In [ ]:
gs.run_command('r.grow.distance', input='water', distance='dist_to_water')
gs.run_command('r.colors', flags='en', map='dist_to_water', color='blues')

#### Distance from protected areas
Like water protected areas can attract urban growth as people enjoy living near scenic areas. We will use raster protected of protected areas we already created, but we will set NULL values to zero. We compute the distance to protected areas with module r.grow.distance and set color table from green to red:

In [ ]:
gs.run_command('r.null', map='protected_areas', setnull=0)
gs.run_command('r.grow.distance', input='protected_areas', distance='dist_to_protected')
gs.run_command('r.colors', map='dist_to_protected', color='gyr', flags='e')

#### Forests
Forest may influence urban growth in different ways. On the one hand it might be costly for developments as tree will need to be removed. On the other hand, people may want to live near forests. We calculate the percentage forest for a 0.5 km pixel to test forest influence, see NLCD categories Forest. We derive the percentage of forest also for year 1992.

In [ ]:
gs.mapcalc("forest = if(landuse_2011 >= 41 && landuse_2011 <= 43, 1, 0)")
gs.mapcalc("forest_1992 = if(landuse_1992 >= 40 && landuse_1992 <= 43, 1, 0)")
gs.run_command('r.neighbors', flags='c', input='forest', output='forest_smooth', size=15, method='average')
gs.run_command('r.neighbors', flags='c', input='forest_1992', output='forest_1992_smooth', size=15, method='average')
gs.run_command('r.colors', map='forest_smooth,forest_1992_smooth', color='ndvi')

#### Distance to main urban centers
Concentrated population and economic activities around main urban centers are important drivers of development. To calculate this influence we will compute distance to urban centers with more than 200,000 inhabitants:

In [ ]:
gs.run_command('r.grow.distance', input='city_center', distance='dist_to_city_center')

#### Road density
Dense road construction often occurs in suburban areas where much much development occurs in U.S. cities. To capture this influence we can calculate road density based on the road network. We rasterize roads and use a moving window analysis (r.neighbors) to compute road density for a 1 km area:

In [ ]:
gs.run_command('r.neighbors', flags='c', input='roads', output='road_dens', size=25, method='average')

### Development pressure

Development pressure (computed by r.futures.devpressure) visualized as 3D surface with binary development map draped as color (orange developed, grey undeveloped).
Development pressure is an important variable in the FUTURES model that is always included as a dynamic variable reinforcing the pressure of past development. We assume that urban development in one location will increase the chance of development in near proximity. New simulated development is incorporated into calculating this development pressure every time step that FUTURES simulates urban growth. To estimate the influence that this local development pressure has, we include the development pressure r.futures.devpressure in our initial model estimates. Development pressure is based on number of neighboring developed cells within a specified search distance, weighted by distance. Its special role in the model allows for path dependent feedback between predicted change and change in subsequent steps.

When gamma increases, development influence decreases more rapidly with distance. Size is half the size of the moving window. When gamma is low, local development influences more distant places. We compute development pressure for development in 1992 for Potential model to estimate how much development pressure influenced new development between 1992 and 2011. Then, we compute it also for 2011 for future projections.

In [ ]:
gs.run_command('r.null', map='urban_2011', null=0)
gs.run_command('r.null', map='urban_1992', null=0)
gs.run_command('r.futures.devpressure', flags='n', input='urban_1992', output='devpressure_30_05_01_92 ',
               method='gravity', size=30, gamma=0.5, scaling_factor=0.1)
gs.run_command('r.futures.devpressure', flags='n', input='urban_2011', output='devpressure_30_05_01_2011',
               method='gravity', size=30, gamma=0.5, scaling_factor=0.1)

In [ ]:
# end the GRASS session
os.remove(rcfile)